In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from shutil import which
import time
from bs4 import BeautifulSoup
from lxml import etree


In [13]:
class nude():
    def __init__(self, items = list(), headless = False):
        
        self.items = items
        self.headless = headless
    
    def scrap(self, category):
        chrome_options = Options()

        if self.headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument("--disable-extensions")

        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)

#         chrome_path = "C:\Users\chaitanya\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
        global driver
        driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver" ,options=chrome_options)
        driver.get("https://us.nudeglass.com/")

        time.sleep(1)
        try:
            driver.find_element(By.XPATH, "//a[@id='spicegems_cr_btn_no']").click() 
        except:
            print('No country change')

        main_page = driver.current_url
        time.sleep(1)
        
        driver.get(str(main_page)+category)
        url = driver.current_url
        num = 2
        
        while num<=5:
            try:
                # scroll()
                product_url = driver.find_elements(By.XPATH, "//h2[@class = 'ProductItem__Title Heading']/a[@href]")
                for product in product_url:
                    product_details = {}
                    href = product.get_attribute("href")
                    driver.execute_script("window.open('" +href +"');")
                    # switch to new window
                    driver.switch_to.window(driver.window_handles[1])

                    try:
                        name = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//h1[@class = 'ProductMeta__Title Heading u-h2']")))
                        product_details['name'] = name.text
                    except:
                        product_details['name'] = None

                    try:
                        price = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "//span[@class = 'ProductMeta__Price Price Text--subdued u-h4']")))
                        product_details['price'] = price.text
                    except:
                        product_details['price'] = None

                    try:
                        designer = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "(//div[@class = 'ProductMeta__Description Rte']/p)[1]")))
                        product_details['designer'] = designer.text
                    except:
                        product_details['designer'] = None

                    try:
                        description = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "(//div[@class = 'ProductMeta__Description Rte']/p)[2]")))
                        product_details['product description'] = description.text
                    except:
                        product_details['product description'] = None

                    try:
                        avaliablity = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.XPATH, "//button[@class = 'ProductForm__AddToCart Button Button--primary Button--full']/span")))
                        product_details['avaliablity'] = avaliablity.text
                    except:
                        product_details['avaliablity'] = "Sold Out"

                    try:
                        tech_description = driver.find_elements(By.XPATH, "//div[@class = 'ProductMeta__Description Rte']/ul/li")
                        technical_description = []
                        for tech in tech_description:
                            technical_description.append(tech.text) 
                        product_details['technical description'] = '\n'.join(technical_description)
                    except:
                        product_details['technical description'] = None

                    try:
                        options = driver.find_elements(By.XPATH, "//select[@class='form-control-gd']/option")
                        color = []
                        for opt in options:
                            color.append(opt.text)
                        product_details['options'] = color
                    except:
                        product_details['options'] = None

                    try:
                        images = driver.find_elements(By.XPATH, "//div[@class='Product__SlideshowNav Product__SlideshowNav--thumbnails']/div[@class = 'Product__SlideshowNavScroller']/a/img")
                        image = []
                        if len(options) >=2:
                            for opt in images[:len(options)]:
                                image.append(opt.get_attribute("src")) 
                            product_details['variant_image'] = image
                            
                        else:
                            image = driver.find_element(By.XPATH, "//div/img[@class = 'Image--fadeIn lazyautosizes Image--lazyLoaded']")
                            image_url = image.get_attribute("src")
                            product_details['variant_image'] = image_url
                    except:
                       
                        product_details['variant_image'] = None

                    try:
                        if len(options)>=2:
                            skus = driver.find_elements(By.XPATH, "//div[@class = 'Select Select--primary']/select/option")
                            sku = []
                            for opt in skus:
                                sku.append(opt.get_attribute("data-sku"))
                            product_details['sku'] = sku
                        else:
                            sku = driver.find_element(By.XPATH, "//div[@class = 'ProductForm__Variants']/input")
                            product_details['sku'] = [sku.get_attribute("data-sku")]
                    
                    except:
                        product_details['sku'] = None


                    try:
                        images = driver.find_elements(By.XPATH, "//div[@class='Product__SlideshowNav Product__SlideshowNav--thumbnails']/div[@class = 'Product__SlideshowNavScroller']/a/img")
                        image = ""
                        for opt in images:
                            image += (opt.get_attribute("src"))+ "\n"
                        product_details['image'] = image
                    except:
                        product_details['image'] = None

                    self.items.append(product_details)
                    #close the new window
                    driver.close()
                    #back to main window
                    driver.switch_to.window(driver.window_handles[0])

            except:
                break


            next_page_url = str(url) + f"?page={num}"
            num += 1
            try:
                driver.get(next_page_url)
            except:
                break

        return self.items
    
    def save_csv(self, category):
        json_file = scrap(category)
        df = pd.DataFrame(json_file)
        df = df.explode(['options', 'sku'])
        df.reset_index(drop=True, inplace=True)

        try:
            os.mkdir("nudeCSV")
        except:
            pass
        
        try:
            df.to_csv("nudeCSV/nudeglass.csv")
        except:
            print("csv file already exists")






In [14]:
data_json = []
start = nude(data_json, False)
categories = ['collections/all-tableware', 'collections/all-home-living', 'collections/all-drinkware']
for category in categories:
    data = start.scrap(category)


C:\Users\CHAITA~1\AppData\Local\Temp/ipykernel_7228/1378130380.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver_win32/chromedriver" ,options=chrome_options)


WebDriverException: Message: Service chromedriver_win32/chromedriver unexpectedly exited. Status code was: 3221225794


In [23]:
len(data)

410

In [24]:
data

[{'name': 'Ecrin\nLidded Vessel Wide',
  'price': '$132',
  'designer': 'Designed by Sebastian Herkner',
  'product description': 'Part of the NUDE Ecrin series designed by Sebastian Herkner, this medium-sized container offers an elevated sense of sophistication and elegance. Hand-crafted from crystalline glass, this precious piece stands out with its commanding, heavy base, a curved profile and a colour-infused lid, which recalls the shape of the Ritten rock formations – an impressive natural monument in South Tyrol.',
  'avaliablity': 'ADD TO CART',
  'technical description': '6.5" / 4.6"\nHandmade\nLead-free crystal\nSmooth orb-like lid\nDue to handmade craft, color may vary slightly',
  'options': ['Petroleum Green', 'Amber'],
  'variant_image': ['https://cdn.shopify.com/s/files/1/0096/0767/6004/products/1105402-92762-Ecrin-Storage_box_medium_with_amber_lid-PL-1_160x.jpg?v=1643799679',
   'https://cdn.shopify.com/s/files/1/0096/0767/6004/products/1105405-92762-Ecrin-Storage_box_med

In [79]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data, )
df.head()


,name,price,designer,product description,avaliablity,technical description,options,variant_image,sku,image
0,Ecrin\nLidded Vessel Wide,$132,Designed by Sebastian Herkner,Part of the NUDE Ecrin series designed by Seba...,ADD TO CART,"6.5"" / 4.6""\nHandmade\nLead-free crystal\nSmoo...","[Petroleum Green, Amber]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92762-1105405, 92762-1105402]",https://cdn.shopify.com/s/files/1/0096/0767/60...
1,Ecrin-Lidded Vessel Large,$154,Designed by Sebastian Herkner,Designed by Sebastian Herkner as part of the N...,ADD TO CART,"5.7"" / 7.2""\nHandmade\nLead-free crystal\nSmoo...","[Petroleum Green, Caramel]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92761-1105404, 92761-1105401]",https://cdn.shopify.com/s/files/1/0096/0767/60...
2,Nest\nCandy Box Large,$91,Design by Pentagon Design,This NUDE Nest candy box serves up progressive...,ADD TO CART,"20.25 fl oz / 5.91"" / 6""\nHandmade\nLead-free ...","[Cobalt Blue, Caramel, Clear, Pink]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92356-1101344, 92356-1101347, 92356-1027133, ...",https://cdn.shopify.com/s/files/1/0096/0767/60...
3,Ecrin\nLidded Vessel Small,$79,Design by Sebastian Herkner,This small NUDE Ecrin container is practical a...,Sold Out,"3.7"" / 4.9""\nHandmade\nLead-free crystal\nSmoo...","[Petroleum Green, Red]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92760-1105403, 92760-1105400]",https://cdn.shopify.com/s/files/1/0096/0767/60...
4,Olea\nOil and Vinegar Bottle,$72,Design by Alejandro Ruiz,This NUDE Olea oil and vinegar bottle is equal...,ADD TO CART,"7.5 fl oz / 3.94"" / 4.5""\nHandmade\nLead-free ...",[],https://cdn.shopify.com/s/files/1/0096/0767/60...,[92548-1050308],https://cdn.shopify.com/s/files/1/0096/0767/60...


In [102]:
df.image[5]

'https://cdn.shopify.com/s/files/1/0096/0767/6004/products/Plain_-_Nest_Box_Caramel_-_92356_160x.jpg?v=1573116195\nhttps://cdn.shopify.com/s/files/1/0096/0767/6004/products/Plain_-_Nest_Box_Cobalt_-_92356_160x.jpg?v=1573116195\nhttps://cdn.shopify.com/s/files/1/0096/0767/6004/products/Plain_-_Nest_Candy_Box_M_Clear_-_92356_v1_160x.jpg?v=1571710875\nhttps://cdn.shopify.com/s/files/1/0096/0767/6004/products/Plain_-_Nest_Candy_Box_M_Pink_-_92356_160x.jpg?v=1571710875\nhttps://cdn.shopify.com/s/files/1/0096/0767/6004/products/duzenlemeler_0003_Lifestyle_-_Nest_Candy_Box_M_Pink_-_92356_160x.jpg?v=1571710875\nhttps://cdn.shopify.com/s/files/1/0096/0767/6004/products/N_3482_160x.jpg?v=1630054595\nhttps://cdn.shopify.com/s/files/1/0096/0767/6004/products/N_3583_c77cfaa0-ec20-462f-86b1-90f6ae70fdf5_160x.jpg?v=1630054595\n'

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   410 non-null    object
 1   price                  410 non-null    object
 2   designer               410 non-null    object
 3   product description    409 non-null    object
 4   avaliablity            410 non-null    object
 5   technical description  410 non-null    object
 6   options                410 non-null    object
 7   variant_image          410 non-null    object
 8   sku                    410 non-null    object
 9   image                  410 non-null    object
dtypes: object(10)
memory usage: 32.2+ KB


In [81]:
df['options'] = df['options'].apply(lambda d: d if len(d)>0 else None)

In [82]:
df['sku'] = df['sku'].apply(lambda d: d[0] if len(d)==1 else d)


In [83]:
df

,name,price,designer,product description,avaliablity,technical description,options,variant_image,sku,image
0,Ecrin\nLidded Vessel Wide,$132,Designed by Sebastian Herkner,Part of the NUDE Ecrin series designed by Seba...,ADD TO CART,"6.5"" / 4.6""\nHandmade\nLead-free crystal\nSmoo...","[Petroleum Green, Amber]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92762-1105405, 92762-1105402]",https://cdn.shopify.com/s/files/1/0096/0767/60...
1,Ecrin-Lidded Vessel Large,$154,Designed by Sebastian Herkner,Designed by Sebastian Herkner as part of the N...,ADD TO CART,"5.7"" / 7.2""\nHandmade\nLead-free crystal\nSmoo...","[Petroleum Green, Caramel]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92761-1105404, 92761-1105401]",https://cdn.shopify.com/s/files/1/0096/0767/60...
2,Nest\nCandy Box Large,$91,Design by Pentagon Design,This NUDE Nest candy box serves up progressive...,ADD TO CART,"20.25 fl oz / 5.91"" / 6""\nHandmade\nLead-free ...","[Cobalt Blue, Caramel, Clear, Pink]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92356-1101344, 92356-1101347, 92356-1027133, ...",https://cdn.shopify.com/s/files/1/0096/0767/60...
3,Ecrin\nLidded Vessel Small,$79,Design by Sebastian Herkner,This small NUDE Ecrin container is practical a...,Sold Out,"3.7"" / 4.9""\nHandmade\nLead-free crystal\nSmoo...","[Petroleum Green, Red]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[92760-1105403, 92760-1105400]",https://cdn.shopify.com/s/files/1/0096/0767/60...
4,Olea\nOil and Vinegar Bottle,$72,Design by Alejandro Ruiz,This NUDE Olea oil and vinegar bottle is equal...,ADD TO CART,"7.5 fl oz / 3.94"" / 4.5""\nHandmade\nLead-free ...",None,https://cdn.shopify.com/s/files/1/0096/0767/60...,92548-1050308,https://cdn.shopify.com/s/files/1/0096/0767/60...
...,...,...,...,...,...,...,...,...,...,...
405,Ghost Zero Belly\nSet of 2 Glasses,$124,Design by NUDE Design Team,Ghost Zero set of two Belly glasses exudes sim...,ADD TO CART,"15.89 fl oz / 4.5"" / 3.75""\nHandmade\nLead-fre...",None,https://cdn.shopify.com/s/files/1/0096/0767/60...,22609-1107948,https://cdn.shopify.com/s/files/1/0096/0767/60...
406,Refine\nSet of 2 White Wine Glasses,$33,Design by Nude Design Team,This NUDE Refine set of white wine glasses com...,ADD TO CART,"10.75 fl oz / 3.19"" / 7.48""\nLead-free crystal",None,https://cdn.shopify.com/s/files/1/0096/0767/60...,67090-1080830,https://cdn.shopify.com/s/files/1/0096/0767/60...
407,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...","[Clear, Smoke, Steel Blue]",[https://cdn.shopify.com/s/files/1/0096/0767/6...,"[22387-1095624, 22387-1116681, 22387-1095626]",https://cdn.shopify.com/s/files/1/0096/0767/60...
408,Neo\nSet of 2 Glasses,$53,Design by Pentagon Design,Perfectly discrete and powerful in its design-...,ADD TO CART,"8.79 fl oz / 3.35"" / 2.36""\nHandmade\nLead-fre...",None,https://cdn.shopify.com/s/files/1/0096/0767/60...,22288-1076997,


In [91]:
df = df.explode(['options', 'sku', 'variant_image'])

In [92]:
df.reset_index(drop=True, inplace=True)


In [93]:
df

,name,price,designer,product description,avaliablity,technical description,options,variant_image,sku,image
0,Ecrin\nLidded Vessel Wide,$132,Designed by Sebastian Herkner,Part of the NUDE Ecrin series designed by Seba...,ADD TO CART,"6.5"" / 4.6""\nHandmade\nLead-free crystal\nSmoo...",Petroleum Green,https://cdn.shopify.com/s/files/1/0096/0767/60...,92762-1105405,https://cdn.shopify.com/s/files/1/0096/0767/60...
1,Ecrin\nLidded Vessel Wide,$132,Designed by Sebastian Herkner,Part of the NUDE Ecrin series designed by Seba...,ADD TO CART,"6.5"" / 4.6""\nHandmade\nLead-free crystal\nSmoo...",Amber,https://cdn.shopify.com/s/files/1/0096/0767/60...,92762-1105402,https://cdn.shopify.com/s/files/1/0096/0767/60...
2,Ecrin-Lidded Vessel Large,$154,Designed by Sebastian Herkner,Designed by Sebastian Herkner as part of the N...,ADD TO CART,"5.7"" / 7.2""\nHandmade\nLead-free crystal\nSmoo...",Petroleum Green,https://cdn.shopify.com/s/files/1/0096/0767/60...,92761-1105404,https://cdn.shopify.com/s/files/1/0096/0767/60...
3,Ecrin-Lidded Vessel Large,$154,Designed by Sebastian Herkner,Designed by Sebastian Herkner as part of the N...,ADD TO CART,"5.7"" / 7.2""\nHandmade\nLead-free crystal\nSmoo...",Caramel,https://cdn.shopify.com/s/files/1/0096/0767/60...,92761-1105401,https://cdn.shopify.com/s/files/1/0096/0767/60...
4,Nest\nCandy Box Large,$91,Design by Pentagon Design,This NUDE Nest candy box serves up progressive...,ADD TO CART,"20.25 fl oz / 5.91"" / 6""\nHandmade\nLead-free ...",Cobalt Blue,https://cdn.shopify.com/s/files/1/0096/0767/60...,92356-1101344,https://cdn.shopify.com/s/files/1/0096/0767/60...
...,...,...,...,...,...,...,...,...,...,...
582,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...",Clear,https://cdn.shopify.com/s/files/1/0096/0767/60...,22387-1095624,https://cdn.shopify.com/s/files/1/0096/0767/60...
583,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...",Smoke,https://cdn.shopify.com/s/files/1/0096/0767/60...,22387-1116681,https://cdn.shopify.com/s/files/1/0096/0767/60...
584,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...",Steel Blue,https://cdn.shopify.com/s/files/1/0096/0767/60...,22387-1095626,https://cdn.shopify.com/s/files/1/0096/0767/60...
585,Neo\nSet of 2 Glasses,$53,Design by Pentagon Design,Perfectly discrete and powerful in its design-...,ADD TO CART,"8.79 fl oz / 3.35"" / 2.36""\nHandmade\nLead-fre...",None,https://cdn.shopify.com/s/files/1/0096/0767/60...,22288-1076997,


In [94]:
df_full = df.reindex(columns=[*df.columns.tolist(), 'gtin', 'option(size)', 'option(finish)', \
                   'finish','display_dimensions', 'length', 'width', 'height', 'depth', \
                    'diameter', 'weight', 'arm height', 'seat height', 'table clearance height', \
                    'maximum dimensions', 'swatch_images', 'pdfs', 'installations', 'collection', 'categories', \
                    'made_in', 'voltage', 'bulb_type', 'bulb_text', 'canopy dimensions', 'UL Listed', \
                    'certifications', 'warranty & care', 'position', 'notes'], fill_value= None)

In [95]:
df_full

,name,price,designer,product description,avaliablity,technical description,options,variant_image,sku,image,...,made_in,voltage,bulb_type,bulb_text,canopy dimensions,UL Listed,certifications,warranty & care,position,notes
0,Ecrin\nLidded Vessel Wide,$132,Designed by Sebastian Herkner,Part of the NUDE Ecrin series designed by Seba...,ADD TO CART,"6.5"" / 4.6""\nHandmade\nLead-free crystal\nSmoo...",Petroleum Green,https://cdn.shopify.com/s/files/1/0096/0767/60...,92762-1105405,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ecrin\nLidded Vessel Wide,$132,Designed by Sebastian Herkner,Part of the NUDE Ecrin series designed by Seba...,ADD TO CART,"6.5"" / 4.6""\nHandmade\nLead-free crystal\nSmoo...",Amber,https://cdn.shopify.com/s/files/1/0096/0767/60...,92762-1105402,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ecrin-Lidded Vessel Large,$154,Designed by Sebastian Herkner,Designed by Sebastian Herkner as part of the N...,ADD TO CART,"5.7"" / 7.2""\nHandmade\nLead-free crystal\nSmoo...",Petroleum Green,https://cdn.shopify.com/s/files/1/0096/0767/60...,92761-1105404,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ecrin-Lidded Vessel Large,$154,Designed by Sebastian Herkner,Designed by Sebastian Herkner as part of the N...,ADD TO CART,"5.7"" / 7.2""\nHandmade\nLead-free crystal\nSmoo...",Caramel,https://cdn.shopify.com/s/files/1/0096/0767/60...,92761-1105401,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nest\nCandy Box Large,$91,Design by Pentagon Design,This NUDE Nest candy box serves up progressive...,ADD TO CART,"20.25 fl oz / 5.91"" / 6""\nHandmade\nLead-free ...",Cobalt Blue,https://cdn.shopify.com/s/files/1/0096/0767/60...,92356-1101344,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...",Clear,https://cdn.shopify.com/s/files/1/0096/0767/60...,22387-1095624,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...",Smoke,https://cdn.shopify.com/s/files/1/0096/0767/60...,22387-1116681,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,Parrot\nSet of 2 Water Glasses,$183,Design by Tomas Kral,This NUDE Parrot set of water glasses seamless...,ADD TO CART,"8 fl oz / 2.76"" / 3.94""\nHandmade\nLead-free c...",Steel Blue,https://cdn.shopify.com/s/files/1/0096/0767/60...,22387-1095626,https://cdn.shopify.com/s/files/1/0096/0767/60...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,Neo\nSet of 2 Glasses,$53,Design by Pentagon Design,Perfectly discrete and powerful in its design-...,ADD TO CART,"8.79 fl oz / 3.35"" / 2.36""\nHandmade\nLead-fre...",None,https://cdn.shopify.com/s/files/1/0096/0767/60...,22288-1076997,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
df_full.to_csv("nudeglass_full.csv")

In [32]:
# df.to_excel('nudeglass.xlsx')

In [97]:
df_full.to_excel('nudeglass_full.xlsx')